![Test Case](./test_case_small.png)

In [ ]:
import json

statements = json.loads('[{"type":"assignment","name":"x","sympy":"3*implicit_param_0_0","implicitParams":[{"name":"implicit_param_0_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.0254,"units_valid":true}],"params":["implicit_param_0_0"]},{"type":"assignment","name":"y","sympy":"implicit_param_1_0","implicitParams":[{"name":"implicit_param_1_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.1016,"units_valid":true}],"params":["implicit_param_1_0"]},{"type":"assignment","name":"length","sympy":"sqrt((x)**(2)+(y)**(2))","implicitParams":[],"params":["x","y"]},{"type":"query","sympy":"length","units":"inch","dimensions":[0,1,0,0,0,0,0,0,0],"units_valid":true,"implicitParams":[],"params":["length"]},{"type":"assignment","name":"velocity","sympy":"(length)/(implicit_param_4_0)","implicitParams":[{"name":"implicit_param_4_0","dimensions":[0,0,1,0,0,0,0,0,0],"si_value":10,"units_valid":true}],"params":["length","implicit_param_4_0"]},{"type":"query","sympy":"velocity","units":"","implicitParams":[],"params":["velocity"]},{"type":"query","sympy":"(implicit_param_6_0)/(implicit_param_6_1)","units":"","implicitParams":[{"name":"implicit_param_6_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":10000,"units_valid":true},{"name":"implicit_param_6_1","dimensions":[0,0,1,0,0,0,0,0,0],"si_value":7200,"units_valid":true}],"params":["implicit_param_6_0","implicit_param_6_1"]}]')
statements

In [ ]:
import sympy
from sympy.parsing.latex import parse_latex, LaTeXParsingError

from sympy.physics.units.definitions.dimension_definitions import \
                                mass, length, time, current,\
                                temperature, luminous_intensity,\
                                amount_of_substance, angle, information

from sympy.physics.units.systems.si import dimsys_SI

from sympy.utilities.iterables import topological_sort

# maps from mathjs dimensions object to sympy dimensions
dim_map = {0:mass, 1:length, 2:time, 3:current, 4:temperature, 5:luminous_intensity,
           6:amount_of_substance, 7:angle, 8:information}

inv_dim_map = {str(value.name):key for key, value in dim_map.items()}

# base units as defined by mathjs
base_units = { (0, 0, 0, 0, 0, 0, 0, 0, 0) : '',
               (1, 0, 0, 0, 0, 0, 0, 0, 0) : 'kg',
               (0, 1, 0, 0, 0, 0, 0, 0, 0) : 'm',
               (0, 0, 1, 0, 0, 0, 0, 0, 0) : 'sec',
               (0, 0, 0, 1, 0, 0, 0, 0, 0) : 'ampere',
               (0, 0, 0, 0, 1, 0, 0, 0, 0) : 'kelvin',
               (0, 0, 0, 0, 0, 1, 0, 0, 0) : 'candela',
               (0, 0, 0, 0, 0, 0, 1, 0, 0) : 'mole',
               (1, 1, -2, 0, 0, 0, 0, 0, 0) : 'N',
               (0, 2, 0, 0, 0, 0, 0, 0, 0) : 'm^2',
               (0, 3, 0, 0, 0, 0, 0, 0, 0) : 'm^3',
               (1, 2, -2, 0, 0, 0, 0, 0, 0) : 'J',
               (1, 2, -3, 0, 0, 0, 0, 0, 0) : 'W',
               (1, -1, -2, 0, 0, 0, 0, 0, 0) : 'Pa',
               (0, 0, 1, 1, 0, 0, 0, 0, 0) : 'coulomb',
               (-1, -2, 4, 2, 0, 0, 0, 0, 0) : 'farad',
               (1, 2, -3, -1, 0, 0, 0, 0, 0) : 'V',
               (1, 2, -3, -2, 0, 0, 0, 0, 0) : 'ohm',
               (1, 2, -2, -2, 0, 0, 0, 0, 0) : 'henry',
               (-1, -2, 3, 2, 0, 0, 0, 0, 0) : 'siemens',
               (1, 2, -2, -1, 0, 0, 0, 0, 0) : 'weber',
               (1, 0, -2, -1, 0, 0, 0, 0, 0) : 'tesla',
               (0, 0, -1, 0, 0, 0, 0, 0, 0) : 'Hz',
               (0, 0, 0, 0, 0, 0, 0, 1, 0) : 'rad',
               (0, 0, 0, 0, 0, 0, 0, 0, 1) : 'bits' }

class FuncContainer(object): pass

# map the sympy dimensional dependences to mathjs dimensions
def get_mathjs_units(dimensional_dependencies):
    # print(dimensional_dependencies)

    mathjs_dims = [0]*9

    all_units_recognized = True
    for name, exp in dimensional_dependencies.items():
        dim_index = inv_dim_map.get(name)
        if dim_index is None:
            # this will hapen if the user references a parameter in an equation that has not been defined
            # will eventually want to allow the user to specify the untis for an undefined parameter
            all_units_recognized = False
            break
        mathjs_dims[dim_index] += exp

    if all_units_recognized:
        mathjs_unit_name = base_units.get(tuple(mathjs_dims))

        if mathjs_unit_name is None:
            mathjs_unit_name = ''
            for i, exp in enumerate(mathjs_dims):
                if exp != 0:
                    key = [0]*9
                    key[i] = 1
                    name = base_units.get(tuple(key))
                    if mathjs_unit_name == '':
                        mathjs_unit_name = f'{name}^{exp}'
                    else:
                        mathjs_unit_name = f'{mathjs_unit_name}*{name}^{exp}'
    else:
        mathjs_unit_name = "Dimension Error"

    return mathjs_unit_name


def get_dims(dimensions):
    dims = sympy.Mul(1, *[dim_map[int(i)]**value for i, value in enumerate(dimensions) if value != 0.0])
    return dims

def dimensional_analysis(parameters, final_equality):
    # print(final_equality.rhs)

    # sub parameter dimensions
    parameter_subs = {param['name']:get_dims(param['dimensions']) for param in parameters}
    # print(parameter_subs)
    final_equation = final_equality.rhs.subs(parameter_subs)

    try:
        result = get_mathjs_units(dimsys_SI.get_dimensional_dependencies(final_equation))
    except TypeError:
        result = "Dimension Error"

    return result

class NoEquality(Exception):
    pass

class ParameterError(Exception):
    pass

class DuplicateAssignment(Exception):
    pass

class ReferenceCycle(Exception):
    pass

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def evaluate_equations(parameters, equations):
    # debug printing, latex sympy
    for equation in equations:
        print(equation['formula'])

    equalities = [None]*len(equations)

    for index, equation in enumerate(equations):
        try:
            equalities[index] = parse_latex(equation['formula'])
            if not isinstance(equalities[index], sympy.Eq):
                raise NoEquality
        except LaTeXParsingError:
            print(f'Latex parsing error for equation {index}')
            equalities[index] = None
        except NoEquality:
            print(f'Missing equality error for equation {index}')
            equalities[index] = None

    # debug printing, result of parse_latex
    for equality in equalities:
        print(equality)

    try:
        combined_equalities = []
        for i in range(len(equalities)):
            temp_equalities = equalities[0:i+1]
            # sub equations into eachother in order if there are more than one
            undefined_dependency = False
            for i, equality in enumerate(reversed(temp_equalities)):
                if equality is None:
                    undefined_dependency = True
                    break
                if i == 0:
                    final_equality = equality
                else:
                    final_equality = sympy.Eq(final_equality.lhs,
                                            final_equality.rhs.subs({
                                                equality.lhs.name : equality.rhs
                                            }))
            if not undefined_dependency:
                combined_equalities.append(final_equality)
            else:
                combined_equalities.append(None)

        # sub parameter values
        parameter_subs = {param['name']:float(param['si_value']) for param in parameters if param['si_value'] is not None}
        if len(parameter_subs) < len(parameters):
            raise ParameterError
        
        dims = []
        values = []
        for equality in combined_equalities:
            if equality is not None:
                dims.append(dimensional_analysis(parameters, equality))
                value = str(sympy.Eq(equality.lhs, equality.rhs.subs(parameter_subs)).rhs.evalf())
                values.append(value if is_number(value) else '')
            else:
                dims.append('')
                values.append('')
                print('Equation Error')

    except ParameterError:
        print('Parameter Error')
        return None

    return (values, dims)


In [ ]:
def sort_statements(statements):
    defined_params = {}
    for i, statement in enumerate(statements):
        if statement["type"] == "assignment":
            if statement["name"] in defined_params:
                raise DuplicateAssignment
            else:
                defined_params[statement["name"]] = i
            
    vertices = range(len(statements))
    edges = []
    
    for i, statement in enumerate(statements):
        for param in statement["params"]:
            ref_index = defined_params.get(param)
            if ref_index:
                edges.append( (ref_index, i) )
                
    try:
        sort_order = topological_sort((vertices,edges))
    except ValueError:
        print('Reference cycle detected')
        raise ReferenceCycle
        
    return sort_order

In [ ]:
sort_statements(statements)